Requirements 
<ul>
    <li>Your function should account for keyboard distance for all characters (even special characters)</li>
    <li>Your function should account for levenshtein distance as well. If there are more typos, then the score should lean more towards 1 (since it is unlikely for a user to make so many typos)</li>
    <li>Think about if it is more likely to make a horizontal typo vs a vertical typo, you may want to assign a weight to differentiate the typos</li>
    <li>Think about the case when the strings have different lengths and how you should handle it</li>
    <li>Think about if it is necessary to distinguish if the character is already very far away (e.g wikip9dia.org vs wikip0dia.org), both are most likely typosquats, is there a need for a different score? How many keyboard characters away then should I consider it to be not a typo vs not typo?</li>
    <li>Try to think of any other conditions / requirements that I may have missed out, and feel free to suggest any</li>
</ul>

what about swapped letters, one-too-many letters

numbers above qwertyuiop are possible typos, but some may be intended typosquats (i.e. o -> 0; E -> 3 ?)

what about when a user presses 2 keys on accident? e.g. wikoipedia -> presses "o" and "k" when trying to press "k"

are special chars/homoglyphs legal in the url box?

what if they miss a letter?


[python-Levenshtein PyPI](https://pypi.org/project/python-Levenshtein/)

[euclidean distance using numpy (stack overflow)](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) (may help make calculating ED more efficient?)

[top 10 most common TLDs](https://www.statista.com/statistics/265677/number-of-internet-top-level-domains-worldwide/)

[domain name regex](https://medium.com/@vaghasiyaharryk/how-to-validate-a-domain-name-using-regular-expression-9ab484a1b430)


[Prototype 3 string check](https://stackoverflow.com/questions/774316/python-difflib-highlighting-differences-inline)

In [1]:
from math import *
import dnstwist
from tldextract import extract
import pylev as ls
# import Levenshtein as ls
import numpy as np
import difflib as dl
import re
import pandas as pd

## Keyboard Euclidean Distance

In [2]:
keyboard_cartesian = {
                        "1": {"y": -1, "x": 0},
                        "2": {"y": -1, "x": 1},
                        "3": {"y": -1, "x": 2},
                        "4": {"y": -1, "x": 3},
                        "5": {"y": -1, "x": 4},
                        "6": {"y": -1, "x": 5},
                        "7": {"y": -1, "x": 6},
                        "8": {"y": -1, "x": 7},
                        "9": {"y": -1, "x": 8},
                        "0": {"y": -1, "x": 9},
                        "-": {"y": -1, "x": 10},
                        "q": {"y": 0, "x": 0},
                        "w": {"y": 0, "x": 1},
                        "e": {"y": 0, "x": 2},
                        "r": {"y": 0, "x": 3},
                        "t": {"y": 0, "x": 4},
                        "y": {"y": 0, "x": 5},
                        "u": {"y": 0, "x": 6},
                        "i": {"y": 0, "x": 7},
                        "o": {"y": 0, "x": 8},
                        "p": {"y": 0, "x": 9},
                        "a": {"y": 1, "x": 0},
                        "s": {"y": 1, "x": 1},
                        "d": {"y": 1, "x": 2},
                        "f": {"y": 1, "x": 3},
                        "g": {"y": 1, "x": 4},
                        "h": {"y": 1, "x": 5},
                        "j": {"y": 1, "x": 6},
                        "k": {"y": 1, "x": 7},
                        "l": {"y": 1, "x": 8},
                        ";": {"y": 2, "x": 9},
                        "'": {"y": 2, "x": 10},
                        "z": {"y": 2, "x": 0},
                        "x": {"y": 2, "x": 1},
                        "c": {"y": 2, "x": 2},
                        "v": {"y": 2, "x": 3},
                        "b": {"y": 2, "x": 4},
                        "n": {"y": 2, "x": 5},
                        "m": {"y": 2, "x": 6},
                        ",": {"y": 2, "x": 7},
                        ".": {"y": 2, "x": 8},
                        "/": {"y": 2, "x": 9}                   
                     }

def euclidean_distance(a,b):
    X = (keyboard_cartesian[a]['x']-keyboard_cartesian[b]['x'])**2
    Y = (keyboard_cartesian[a]['y']-keyboard_cartesian[b]['y'])**2
    return sqrt(X+Y)

print(euclidean_distance('q', 'w'))
print(euclidean_distance('q', 's'))

1.0
1.4142135623730951


In [3]:
# https://stackoverflow.com/questions/44113335/extract-domain-from-url-in-python

def replace_special_char(char):
    flag = '"!@#$%^&*()+?_=,<>'':\\'
    flag_list = [char for char in flag]
    if char.isalnum()==False and char in flag:
        return 'Z'
    return char
    
# Clean the string first. The extract python library cannot properly extract the domain from URLs with special characters

# 1. make string lower case
# 2. replace all flagged special characters with 'Z'
# 3. extract the domain or TLD
# 4. replace all 'Z' with '!'
# 5. calculate edit distance

def clean_string(url):
    # First make the string lowercase
    url = url.lower()

    # ':' is a flagged character, but if it appears with http or https it is fine
    url = url.replace('https://','')
    url = url.replace('http://','')
    return ''.join([replace_special_char(char) for char in url])
    
def extract_domain_and_tld(url):
    url = clean_string(url)
    tsd, td, tsu = extract(url)
    final = td + '.' + tsu
    return final.replace('Z','!')
    
def extract_tld(url):
    url = clean_string(url)
    tsd, td, tsu = extract(url)
    return tsu.replace('Z','!')  

def extract_sld(url):
    url = clean_string(url)
    tsd, td, tsu = extract(url)
    return td.replace('Z','!')

# Theres no need for this container to be mutable and tuples are faster
# Index 0 - 9 = most popular to least popular
tlds = ("com", "ru", "org", "net", "in", "ir", "au", "uk", "de", "br")

whitelist = ['https://www.wikipedia.org/']


whitelist_domains = []
whitelist_slds = []
for url in whitelist:
    whitelist_domains.append(extract_domain_and_tld(url))
    whitelist_slds.append(extract_sld(url))
typosquat = []
for url in whitelist_domains:
    fuzz = dnstwist.DomainFuzz(url)
    fuzz.generate()
    typosquat.extend([x['domain-name'] for x in fuzz.domains])
    
#Lookups in sets are much more efficient
typosquat = set(typosquat)

#Delete the original whitelisted domains from the blacklist set
typosquat.difference_update(whitelist_domains)

print('Number of typosquatted urls generated: ', len(typosquat))

Number of typosquatted urls generated:  7900


### Prototype 4
- included LD == 1 for further checks
- added new helper function: exceeds_ked_check()
- improved efficiency, removed redundant/repetitive codes

Helper Functions

In [4]:

# Input - 1 URL
# Output - True / False

# e.g contains_special_characters(wikipedia.org) 
# expected output False

# e.g contains_special_characters(wikipedi@.org) 
# expected output True

def contains_special_characters(url):
    return not re.match("^((?!-)[A-Za-z0-9-]{1,}(?<!-)\.)+[A-Za-z0-9]{1,}$", url)


# Helper function to find out if the first TLD is more common than the second TLD

# Input - 2 TLDs
# Output - True / False

# e.g exact_tld_swap(com, org) 
# expected output True

# e.g exact_tld_swap(org, com) 
# expected output False

def tld_more_common(tld1, tld2):    
    tld1_index = 10
    tld2_index = 10
    if tld1 in tlds:
        tld1_index = tlds.index(tld1)
    if tld2 in tlds:
        tld2_index = tlds.index(tld2)
    if tld2_index > tld1_index:
        return True
    elif tld2_index < tld1_index:
        return False


# Helper function to check if "wrong" character is within KED of any of the other supplied characters

# Input - 4 characters
# Output - False

# e.g. exceeds_ked_check("i", "i", "k", "i")
# expected output True

# e.g. exceeds_ked_check("p", "d", "e", "m")
# expected output False
    
def exceeds_ked_check(left, right, correct, wrong):
    result = True
    
    # if the wrong/extra char is at the start of the url, the left will most likely be empty
    if left != "":
        if euclidean_distance(left, wrong) < 1.5:
            result = False
    
    # if the wrong/extra char is at the end of the url, the right will most likely be empty
    if right != "":
        if euclidean_distance(right, wrong) < 1.5:
            result = False
    
    # if the error is an extra char, there wont be a "correct" char
    if correct != "":
        if euclidean_distance(correct, wrong) < 1.5:
            result = False
        
    return result
    
    
# Helper function to perform various euclidean distance checks based off of the lengths of both URLs

# Input - 2 URLs
# Output - dictionary of results

# e.g. edit_check("w1k1pedia.org", "wikipedia.org")
# expected output: {'result': 1, 'reasons_typosquat': ["'1' key and 'i' key are too far apart", "'1' key and 'i' key are too far apart"], 'reasons_typo': []}

def edit_check(sus_url, legit_url):
    result = {"result": 0, "reasons_typosquat": [], "reasons_typo": []}
    
    # retrieving length of both URLs
    sus_len = len(sus_url)
    legit_len = len(legit_url)
    
    # retrieving levenshtein distance of both urls
    ld = ls.levenshtein(sus_url, legit_url)
    
    # if the suspicious is only missing characters, then it's highly likely a typo than a typosquat
    if sus_len == legit_len - 1 and ld == 1 or sus_len == legit_len - 2 and ld == 2:
        result["reasons_typo"].append("Only missing 1-2 characters")
    
    else:
        # creating an object that compares both urls
        seqm = dl.SequenceMatcher(None, sus_url, legit_url)

        # get_opcodes() gets the "differences" between each url, or the steps required for first url to match second url
        for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
            # a : first url
            # b : second url
            # a0, a1 | b0, b1 : index range holding the characters being compared
            # opcode : "equal", "insert", "delete", "replace" -- indicates the action require to turn a to b

            # if a character has to be deleted, means it's an extra character
            if opcode == 'delete':
                
                # extra character(s)
                extra_char = seqm.a[a0:a1]
                
                # if extra characters are side by side
                if len(extra_char) == 2:
                    
                    # extract each extra char
                    extra1 = extra_char[0]
                    extra2 = extra_char[1]

                    left_char = seqm.a[a0-1:a1-2]
                    right_char = seqm.a[a0+2:a1+1]

                    if exceeds_ked_check(left_char, right_char, "", extra1) and exceeds_ked_check(extra1, "", "", extra2):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Extra character(s) ('{}') exceed keyboard euclidean distance boundary".format(extra1))
                        return result
                    elif exceeds_ked_check(left_char, right_char, "", extra2):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Extra character(s) ('{}') exceed keyboard euclidean distance boundary".format(extra2))
                        return result
                    
                # if the extra character is on its own
                else:
                    left_char = seqm.a[a0-1:a1-1]
                    right_char = seqm.a[a0+1:a1+1]
                    
                    if exceeds_ked_check(left_char, right_char, "", extra_char):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Extra character ('{}') exceeds keyboard euclidean distance boundary".format(extra_char))
                        return result
                
            # if a character has to be replaced, means its a swapped character
            elif opcode == 'replace':
                wrong_char = seqm.a[a0:a1]
                correct_char = seqm.b[b0:b1]
                
                # if there are 2 wrong characters side by side
                # therefore, wrong_char == 2, correct char == 2
                if len(wrong_char) == 2 and len(correct_char) == 2:
                    
                    # extract each wrong char, and each correct char for comparison
                    wrong1 = wrong_char[0]
                    wrong2 = wrong_char[1]
                    correct1 = correct_char[0]
                    correct2 = correct_char[1]
                    left_char = seqm.a[a0-1:a1-2]
                    right_char = seqm.a[a0+2:a1+1]
                    
                    if exceeds_ked_check(left_char, correct2, correct1, wrong1) and exceeds_ked_check(wrong1, "", "", wrong2):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong1))
                        return result
                    elif exceeds_ked_check(correct1, right_char, correct2, wrong2):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong2))
                        return result
                
                # if there is an extra char + a wrong char side-by-side, both will be replaced by the 1 correct char
                # therefore, wrong_char == 2, correct_char == 1
                elif len(wrong_char) == 2:
                    
                    # extract each wrong char for comparison
                    wrong1 = wrong_char[0]
                    wrong2 = wrong_char[1]
                    left_char = seqm.a[a0-1:a1-2]
                    right_char = seqm.a[a0+2:a1+1]
                    
                    if exceeds_ked_check(left_char, wrong2, correct_char, wrong1):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Extra/Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong1))
                        return result
                    elif exceeds_ked_check(wrong1, right_char, correct_char, wrong2):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Extra/Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong2))
                        return result
                
                # if there is a wrong char + a missing char side-by-side, the wrong char will be replace by the correct char + the missing char
                # therefore, wrong_char == 1, correct char == 2
                elif len(correct_char) == 2:
                    
                    # extract each correct char for comparison
                    correct1 = correct_char[0]
                    correct2 = correct_char[1]
                    
                    if exceeds_ked_check("", "", correct1, wrong_char) and exceeds_ked_check("", "", correct2, wrong_char):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong_char))
                        return result
                
                # if there is only one wrong char, to be replaced by one correct char
                # therefore, wrong_char == 1, correct char == 1
                else:
                    if exceeds_ked_check("", "", correct_char, wrong_char):
                        result["result"] = 1
                        result["reasons_typosquat"].append("Wrong character ('{}') exceeds keyboard euclidean distance boundary".format(wrong_char))
                        return result
                    
        result["reasons_typo"].append("Extra/Wrong characters within keyboard euclidean distance boundary")
        
    return result

Main Program

In [5]:
def is_typo(sus_url, legit_url):
    # preparing 
    result = {"suspicious url": sus_url, "original url": legit_url, "result": 0, "reasons_typosquat": [], "reasons_typo": []}
    
    # extracting SLD from both URLs
    sus_sld = extract_sld(sus_url)
    legit_sld = extract_sld(legit_url)
    
    # extracting TLD from both URLs
    sus_tld = extract_tld(sus_url)
    legit_tld = extract_tld(legit_url)
    
    # retrieving length of both URLs
    sus_len = len(sus_url)
    legit_len = len(legit_url)
    
    # check for illegal special characters
    if contains_special_characters(sus_url):
        result["result"] = 1
        result["reasons_typosquat"].append("Illegal characters found in url")
        return result
    
    # check for exact TLD swap
    if sus_sld == legit_sld and sus_tld != legit_tld:
        if tld_more_common(sus_tld, legit_tld):
            result["reasons_typo"].append("TLD is more common")
            return result
        else:
            result["result"] = 1
            result["reasons_typosquat"].append("TLD is less common")
            return result
    
    # check edit distance
    ld = ls.levenshtein(sus_url, legit_url)
    
    # if only 1 or 2 edits
    if ld == 1 or ld == 2:
        res = edit_check(sus_url, legit_url)
        
        result["reasons_typosquat"].extend(res["reasons_typosquat"])
        result["reasons_typo"].extend(res["reasons_typo"])
        
        if res["result"] == 1:
            result["result"] = 1
            return result
    
    # if 3 or more edits
    else:
        result["result"] = "Inconclusive"
        return result
    
    return result

Test Cases


In [23]:
legit_url = whitelist_domains[0]
test_urls = ["wikipedi@.org", "wikipedia.com", "wikipedia.br", "wikipedi.org", "kipedia.org", "wikipediabb.org", "wwikipediac.org", "wikipedia.bvg", "wikipemnia.org", "wbipedia.org", "wiklped1o.0rg", "wikiped1a.org", "wikipediaa.org", "wlklpedla.org"] 
test_results = []

for url in test_urls:
    test_results.append(is_typo(url, legit_url))
    
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(test_results)
df

,suspicious url,original url,result,reasons_typosquat,reasons_typo
0,wikipedi@.org,wikipedia.org,1,[Illegal characters found in url],[]
1,wikipedia.com,wikipedia.org,0,[],[TLD is more common]
2,wikipedia.br,wikipedia.org,1,[TLD is less common],[]
3,wikipedi.org,wikipedia.org,0,[],[Only missing 1-2 characters]
4,kipedia.org,wikipedia.org,0,[],[Only missing 1-2 characters]
5,wikipediabb.org,wikipedia.org,1,[Extra character(s) ('b') exceed keyboard euclidean distance boundary],[]
6,wwikipediac.org,wikipedia.org,1,[Extra character ('c') exceeds keyboard euclidean distance boundary],[]
7,wikipedia.bvg,wikipedia.org,0,[],[Extra/Wrong characters within keyboard euclidean distance boundary]
8,wikipemnia.org,wikipedia.org,0,[],[Extra/Wrong characters within keyboard euclidean distance boundary]
9,wbipedia.org,wikipedia.org,1,[Wrong character ('b') exceeds keyboard euclidean distance boundary],[]
